In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''
import numpy as np
import tensorflow as tf
from collections import OrderedDict
import tqdm
import matplotlib.pyplot as plt
from plotting import plot_3d, plot_2d
from tf2kfac import tf2KFAC
import tensorflow_probability as tfp
%matplotlib widget

In [2]:
input_data = tf.convert_to_tensor(np.load('data/input.npy').astype(np.float32))
weights = np.load('data/weights.npy').astype(np.float32)
labels = tf.convert_to_tensor(np.load('data/labels.npy').astype(np.int32))
labels = tf.reshape(labels, (-1,))

input_data = tf.convert_to_tensor(input_data)
labels = tf.convert_to_tensor(labels)

input_dim = 2
output_dim = 2




### First lets compare the pure loss computation

In [3]:
input_data = tf.convert_to_tensor(np.load('data/input.npy'), dtype=tf.float32)
weights = tf.convert_to_tensor(np.load('data/weights.npy'), dtype=tf.float32)
labels = tf.convert_to_tensor(np.load('data/labels.npy'), dtype=tf.float32)

In [5]:
input_data = tf.convert_to_tensor(np.load('data/input.npy'))
weights = tf.convert_to_tensor(np.load('data/weights.npy'))
labels = tf.convert_to_tensor(np.load('data/labels.npy'))
labels = tf.reshape(labels, (-1,))

kfac = tf2KFAC(tinv=10,
               lr0=0.005,
               cov_weight=1.,
               cov_moving_weight=1.,
               damping=0.001,
               conv_approx='mg',
               damping_method='factored_tikhonov',
               ft_method='original')


# need a softmax layer to define the output distribution p(y | x) to compute the fisher
print(input_data.shape)
with tf.GradientTape(True) as g:
    g.watch(weights)
    logits = input_data @ weights
    output_dist = tf.nn.softmax(logits)
    neg_log_prob = -tf.math.log(output_dist)
    
    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits))

sensitivities = g.gradient(neg_log_prob, logits)
grads = g.gradient(loss, weights)
print('Loss', loss)
print('Gradients', grads)

Factored Tikhonov damping
(16, 2)
Loss tf.Tensor(1.1128990160589591, shape=(), dtype=float64)
Gradients tf.Tensor(
[[-0.39162274  0.39162274]
 [ 0.08551887 -0.08551887]], shape=(2, 2), dtype=float64)


### KFAC

In [ ]:

dist = tfp.distributions.Normal(0.0, tfsqrt(0.5))

history = []
for i in range(1):
    activations = input_data
    with tf.GradientTape(persistent=True) as g:
        g.watch(weights)
        prediction = input_data@weights
        model_dist = tf.reduce_mean((prediction - target)**2)
        log_prob = - dist.log_prob(prediction)
        loss = 
    sensitivities = g.gradient(log_prob, prediction)
    
    grads = g.gradient(loss, weights)
    
    history.append(np.concatenate([weights.numpy().reshape(-1), loss.numpy()[None]]))
    ng = kfac.compute_updates([activations], [sensitivities], [grads], i)
    weights = weights + ng[0]

    if i % 2 == 0:
        print(i, 'Loss', loss)
history = np.array(history, dtype=np.float32)


In [ ]:
print(kfac.m_aa[0])
print(kfac.m_ss[0])

In [ ]:
plt.close()
plot_3d(history, input_data, target)
plt.show()

In [ ]:
plt.close()
plt.title('TF2 KFAC')
plot_2d(history, input_data, target, (-0.5, 1, -3, 1))
plt.show()